<a href="https://colab.research.google.com/github/krupaanand312jc/EMG_Hand_Movements_NN/blob/main/EMG_Hand_Movements_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/krupaanand312jc/EMG_Hand_Movements_NN

fatal: destination path 'EMG_Hand_Movements_NN' already exists and is not an empty directory.


In [2]:
% cd /content/EMG_Hand_Movements_NN

/content/EMG_Hand_Movements_NN


In [3]:
import numpy as np
import os
from scipy import io
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split

In [4]:
%matplotlib inline

In [5]:
n_classes = 6
n_channels = 1
split_type = 'train'
train_days = [1, 2, 3]
test_days = [1, 2, 3]

if split_type == 'train':
        split = train_days
else:
        split = test_days

# Assign numeric label to categories:
#
# cyl = 1
# hook = 2
# lat = 3
# palm = 4
# spher = 5
# tip = 6
#
    
labels = np.concatenate(
        (
            [[class_id for _ in range(100 * len(split))] for class_id in range(1, n_classes + 1)]
        )
    )

In [6]:
data1 = io.loadmat('/content/EMG_Hand_Movements_NN/Dataset/sEMG_Basic_Hand_movements_upatras/Database 2/male_day_1.mat')
data2 = io.loadmat('/content/EMG_Hand_Movements_NN/Dataset/sEMG_Basic_Hand_movements_upatras/Database 2/male_day_2.mat')
data3 = io.loadmat('/content/EMG_Hand_Movements_NN/Dataset/sEMG_Basic_Hand_movements_upatras/Database 2/male_day_1.mat')

cyl_day1 = data1['cyl_ch2']
hook_day1 = data1['hook_ch2']
tip_day1 = data1['tip_ch2']
palm_day1 = data1['palm_ch2']
spher_day1 = data1['spher_ch2']
lat_day1 = data1['lat_ch2']

cyl_day2 = data2['cyl_ch2']
hook_day2 = data2['hook_ch2']
tip_day2 = data2['tip_ch2']
palm_day2 = data2['palm_ch2']
spher_day2 = data2['spher_ch2']
lat_day2 = data2['lat_ch2']

cyl_day3 = data3['cyl_ch2']
hook_day3 = data3['hook_ch2']
tip_day3 = data3['tip_ch2']
palm_day3 = data3['palm_ch2']
spher_day3 = data3['spher_ch2']
lat_day3 = data3['lat_ch2']

In [7]:
X = np.vstack([cyl_day1, cyl_day2, cyl_day3,
               hook_day1, hook_day2, hook_day3,
               lat_day1, lat_day2, lat_day3,
               palm_day1, palm_day2, palm_day3,
               spher_day1, spher_day2, spher_day3,
               tip_day1, tip_day2, tip_day3])

# Drop last 4 data points to more easily subdivide into layers
X = np.delete(X, [2496, 2497, 2498, 2499], 1)

X = X.reshape(1800, 2496, 1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.4, random_state=42)

In [9]:
# Standardize train and test
X_train = (X_train - np.mean(X_train, axis=0)[None, :, :]) / np.std(X_train, axis=0)[None, :, :]
X_test = (X_test - np.mean(X_test, axis=0)[None, :, :]) / np.std(X_test, axis=0)[None, :, :]

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [11]:
def one_hot(labels, n_class=6):
    """
    One-hot encoding.
    :param labels: Labels to encode.
    :param n_class: Number of classes.
    :return: One-hot encoded labels.
    """
    expansion = np.eye(n_class)
    y = expansion[:, labels-1].T

    assert y.shape[1] == n_class, "Wrong number of labels!"

    return y


y_train = one_hot(y_train)
y_val = one_hot(y_val)
y_test = one_hot(y_test)

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [13]:
# The following were determined through iterative trial and error:
batch_size = 100
seq_len = 2496
learning_rate = 0.00001
epochs = 2500

n_classes = 6
n_channels = 1

In [14]:
model = tf.keras.models.Sequential([
  # (batch, 2496, 1) --> (batch, 1248, 2)
  tf.keras.layers.Conv1D(filters=2, kernel_size=1, strides=1, padding='same', activation = 'relu', input_shape = (2496,1)),
  tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),
  # (batch, 1248, 2) --> (batch, 624, 4)
  tf.keras.layers.Conv1D(filters=4, kernel_size=1, strides=1, padding='same', activation = 'relu'),
  tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),
  # (batch, 624, 4) --> (batch, 312, 8)
  tf.keras.layers.Conv1D(filters=8, kernel_size=1, strides=1, padding='same', activation = 'relu'),
  tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),
  # (batch, 312, 8) --> (batch, 156, 16)
  tf.keras.layers.Conv1D(filters=16, kernel_size=1, strides=1, padding='same', activation = 'relu'),
  tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),
  # (batch, 156, 16) --> (batch, 78, 32)
  tf.keras.layers.Conv1D(filters=32, kernel_size=1, strides=1, padding='same', activation = 'relu'),
  tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),
  # (batch, 78, 32) --> (batch, 39, 64)
  tf.keras.layers.Conv1D(filters=64, kernel_size=1, strides=1, padding='same', activation = 'relu'),
  tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),

  tf.keras.layers.Flatten(),

  tf.keras.layers.Dense(156, activation='relu'),

  tf.keras.layers.Dropout(0.5),

  tf.keras.layers.Dense(18, activation='relu'),

  tf.keras.layers.Dropout(0.5),

  tf.keras.layers.Dense(n_classes, activation='softmax')

]) 

model.compile(tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size = batch_size, epochs=epochs)

Streaming output truncated to the last 5000 lines.
Epoch 1/2500
9/9 [==============================] - 4s 99ms/step - loss: 1.7904 - accuracy: 0.1827 - val_loss: 1.7683 - val_accuracy: 0.2148
Epoch 2/2500
9/9 [==============================] - 0s 15ms/step - loss: 1.7986 - accuracy: 0.1975 - val_loss: 1.7647 - val_accuracy: 0.2037
Epoch 3/2500
9/9 [==============================] - 0s 17ms/step - loss: 1.7900 - accuracy: 0.1926 - val_loss: 1.7621 - val_accuracy: 0.1963
Epoch 4/2500
9/9 [==============================] - 0s 16ms/step - loss: 1.7962 - accuracy: 0.1840 - val_loss: 1.7594 - val_accuracy: 0.1963
Epoch 5/2500
9/9 [==============================] - 0s 15ms/step - loss: 1.7846 - accuracy: 0.1704 - val_loss: 1.7572 - val_accuracy: 0.2037
Epoch 6/2500
9/9 [==============================] - 0s 16ms/step - loss: 1.7918 - accuracy: 0.1691 - val_loss: 1.7541 - val_accuracy: 0.2185
Epoch 7/2500
9/9 [==============================] - 0s 16ms/step - loss: 1.7797 - accuracy: 0.2037 - va

In [15]:
model.evaluate(X_test, y_test, batch_size=batch_size)

8/8 [==============================] - 0s 22ms/step - loss: 0.8501 - accuracy: 0.7125


[0.8500847816467285, 0.7124999761581421]

In [16]:
y_pred = model.predict(X_test)

In [17]:
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)

In [18]:
#importing confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

Confusion Matrix

[[ 98   0   0   0  15   0]
 [  3  92  13  12   7   9]
 [  0  10  74  15   0  15]
 [  1  14  10  66   2  15]
 [ 26   8   2   0 103   0]
 [  1   6  12  10   1  80]]


In [19]:
#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['Cylindrical', 'Hook', 'Lateral', 'Palmer', 'Spherical', 'tip']))


Accuracy: 0.71

Micro Precision: 0.71
Micro Recall: 0.71
Micro F1-score: 0.71

Macro Precision: 0.71
Macro Recall: 0.71
Macro F1-score: 0.71

Weighted Precision: 0.71
Weighted Recall: 0.71
Weighted F1-score: 0.71

Classification Report

              precision    recall  f1-score   support

 Cylindrical       0.76      0.87      0.81       113
        Hook       0.71      0.68      0.69       136
     Lateral       0.67      0.65      0.66       114
      Palmer       0.64      0.61      0.63       108
   Spherical       0.80      0.74      0.77       139
         tip       0.67      0.73      0.70       110

    accuracy                           0.71       720
   macro avg       0.71      0.71      0.71       720
weighted avg       0.71      0.71      0.71       720



In [20]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: saved_model/1/assets


In [21]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()